In [1]:
# set working directory
import os
os.chdir('..')
from py.firth import firth_logistic_regression
import pandas as pd
import numpy as np
print(os.getcwd())
print(os.listdir())

c:\Users\bbessell\Projects\GitHub\pyZINQ
['.git', '.gitignore', 'License.md', 'py', 'README.md', 'refs', 'tests']


In [7]:
data = pd.read_csv("refs/uti_data.csv", delimiter=";").drop(columns=["Unnamed: 0"])
data.head()

covariates = ["age", "oc", "vic", "vicl", "vis", "dia"]
X_ = data[covariates].to_numpy().astype(np.float64)
y_ = data["case"].to_numpy().astype(np.float64)

print(X_.shape)

beta, std_err, fitll, wald, pvals = firth_logistic_regression(y_, X_, tol=1e-8, test='lrt')
summary = pd.DataFrame({
    "covariate": ["intercept"] + covariates,
    "beta": beta,
    "std_err": std_err,
    "fitll": fitll,
    "wald": wald,
    "pvals": pvals
})
print(summary)

(239, 6)
0
1
2
3
4
5
6
   covariate      beta   std_err       fitll       wald         pvals
0  intercept  0.120255  0.485542 -138.455068  -0.000000  1.000000e+00
1        age -1.105989  0.423662 -138.455068   7.885723  4.982656e-03
2         oc -0.068815  0.443794 -138.455068   0.002921  9.568986e-01
3        vic  2.268876  0.548417 -138.455068  24.122429  9.040123e-07
4       vicl -2.111412  0.543084 -138.455068  20.088726  7.393119e-06
5        vis -0.788319  0.417368 -138.455068   3.821936  5.058576e-02
6        dia  3.096556  1.675363 -138.455068   9.435747  2.127964e-03


In [ ]:
# output of R's logistf package:
'''
logistf(formula = case ~ age + oc + vic + vicl + vis + dia, data = sex2)

Model fitted by Penalized ML
method
                   coef  se(coef) lower 0.95  upper 0.95       Chisq            p
(Intercept)  0.12025405 0.4763429 -0.8133609  1.05386904  0.06373235 8.006912e-01      1
age         -1.10598131 0.4149021 -1.9191744 -0.29278818  7.10565893 7.684097e-03      1
oc          -0.06881673 0.4344026 -0.9202301  0.78259664  0.02509593 8.741283e-01      1
vic          2.26887464 0.5384872  1.2134590  3.32429026 17.75293469 2.515292e-05      1
vicl        -2.11140817 0.5320395 -3.1541864 -1.06862995 15.74913385 7.232104e-05      1
vis         -0.78831694 0.4089620 -1.5898677  0.01323384  3.71565877 5.390435e-02      1
dia          3.09601166 1.5052197  0.1458353  6.04618801  4.23063350 3.970062e-02
'''

In [5]:
# unit test
correct_betas = [0.12025405, -1.10598131, -0.06881673, 2.26887464, -2.11140817, -0.78831694, 3.09601166]
correct_pvals = [8.006912e-01, 7.684097e-03, 8.741283e-01, 2.515292e-05, 7.232104e-05, 5.390435e-02, 3.970062e-02]

print("diff betas: ", beta - correct_betas)
print("diff pvals: ", pvals - correct_pvals)

assert np.allclose(beta, correct_betas, atol=1e-3)
assert np.allclose(pvals, correct_pvals, atol=5e-2)
print("pass")



diff betas:  [ 1.07547291e-06 -7.31802781e-06  1.28657029e-06  1.49313811e-06
 -3.60615652e-06 -1.72431510e-06  5.44770210e-04]


TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [5]:
import timeit
print(timeit.timeit(lambda: firth_logistic_regression(y_, X_, test='wald'), number=100))

22.722841800001333


In [7]:
(23/100)*15000/60/60

0.9583333333333334

In [238]:
# WIP WIP WIP WIP

import numpy as np
from numba import njit
from scipy import special, linalg
import warnings

@njit
def logit(p):
    return np.log(p / (1 - p))

@njit
def logistic(x, eps=1e-5):
    """ Compute logistic function, clipping the output to be within (eps, 1-eps). """
    # Add clipping to avoid values exactly at 0 or 1
    return np.clip(1 / (1 + np.exp(-x)), eps, 1 - eps)

@njit
def compute_log_likelihood(beta, X, y):
    logits = X @ beta
    log_likelihood = np.sum(y * logits - np.log(1 + np.exp(logits)))
    return log_likelihood

@njit
def firth_penalty(log_likelihood, H):
    sign, logdet = np.linalg.slogdet(-H)
    return -(log_likelihood + 0.5 * logdet)

@njit
def compute_fisher_information(X, pi, eps=np.finfo(float).eps):
    """ Compute Fisher information matrix, ensuring numerical stability. """
    # Use an epsilon value to ensure pi * (1 - pi) does not result in zero rows
    W = np.diag(np.clip(pi * (1 - pi), eps, None))  # Add a small epsilon
    H = X.T @ (W @ X)
    return np.linalg.pinv(H)

@njit
def compute_score(X, y, pi, H):
    root_diag_h = np.sqrt(np.diag(X @ (np.linalg.pinv(-H) @ X.T)))
    delta_pi = root_diag_h * (0.5 - pi)
    U = X.T @ (y - pi + delta_pi)
    return U

@njit
def compute_standard_errors(inverse_fisher_info):
    return np.sqrt(np.diag(inverse_fisher_info))

@njit
def compute_wald_test(beta, standard_errors):
    wald_stats = np.square(beta / standard_errors)
    p_values = 2 * (1 - norm.cdf(np.abs(beta / standard_errors)))  # two-tailed

    return wald_stats, p_values

@njit
def compute_likelihood_ratio_test(full_ll, reduced_ll):
    lr_stat = -2 * (reduced_ll - full_ll)
    p_value = chi2.sf(lr_stat, df=1)  # df=1 for a single parameter test
    return lr_stat, p_value


def firth_logistic_regression(y, X, max_iter=1000, tol=1e-6, test='wald'):
    N, p = X.shape
    beta = np.zeros(p+1)
    X = np.hstack((np.ones((N, 1)), X))  # Adding a column for the intercept

    for i in range(max_iter):
        pi = logistic(X @ beta)
        H = compute_fisher_information(X, pi)
        ll = compute_log_likelihood(beta, X, y)

        if firth_penalty(ll, H) < tol:
            break

        U = compute_score(X, y, pi, H)

        inverse_H = linalg.pinv(-H)  # Inverting the Hessian matrix
        print("Shape of inverse_H:", inverse_H.shape)
        print("Shape of U:", U.shape)
        delta_beta = inverse_H @ U
        print("Shape of delta_beta:", delta_beta.shape)
        beta = beta + delta_beta

        if np.max(np.abs(delta_beta)) < tol:
            break
    
    se = None

    if test == 'null_model':
        return None, None, ll, None, None
    elif test == 'wald':
        se = compute_standard_errors(np.linalg.pinv(-H))
        stats, pvals = compute_wald_test(beta, se)
    elif test == 'lrt':
        null_X = np.delete(X, i, axis=1)
        _, _, null_ll = firth_logistic_regression(y, null_X, test='null_model')
        stats, pvals = compute_likelihood_ratio_test(ll, null_ll)
    else:
        raise ValueError('Invalid test type')

    se = compute_standard_errors(np.linalg.pinv(-H)) if se is None else se
    
    return beta, se, ll, stats, pvals

In [4]:
data = pd.read_csv("../refs/uti_data.csv", delimiter=";").drop(columns=["Unnamed: 0"])
data.head()

covariates = ["age", "oc", "vic", "vicl", "vis", "dia"]
X_ = data[covariates].to_numpy().astype(np.float64)
y_ = data["case"].to_numpy().astype(np.float64)

beta, std_err, fitll, wald, pvals = firth_logistic_regression(y_, X_, tol=1e-8, test='wald')
summary = pd.DataFrame({
    "covariate": ["intercept"] + covariates,
    "beta": beta,
    "std_err": std_err,
    "fitll": fitll,
    "wald": wald,
    "pvals": pvals
})
print(summary)

FileNotFoundError: [Errno 2] No such file or directory: '../refs/uti_data.csv'

In [ ]:
'''
Model fitted by Penalized ML
method
                   coef  se(coef) lower 0.95  upper 0.95       Chisq            p
(Intercept)  0.12025405 0.4763429 -0.8133609  1.05386904  0.06373235 8.006912e-01      1
age         -1.10598131 0.4149021 -1.9191744 -0.29278818  7.10565893 7.684097e-03      1
oc          -0.06881673 0.4344026 -0.9202301  0.78259664  0.02509593 8.741283e-01      1
vic          2.26887464 0.5384872  1.2134590  3.32429026 17.75293469 2.515292e-05      1
vicl        -2.11140817 0.5320395 -3.1541864 -1.06862995 15.74913385 7.232104e-05      1
vis         -0.78831694 0.4089620 -1.5898677  0.01323384  3.71565877 5.390435e-02      1
dia          3.09601166 1.5052197  0.1458353  6.04618801  4.23063350 3.970062e-02
'''

In [ ]:

# A working but slow implementation of Firth logistic regression
'''
import numpy as np
import pandas as pd
import statsmodels.api as sm
import sys
from numba import jit, njit
from scipy.stats import chi2, norm


@njit
def firth_likelihood(loglike_val, H):
    return -(loglike_val + .5*np.log(np.linalg.det(-H)))


@njit
def fit_firth_numba(betas, hessian, y, X):
    pi = 1 / (  1 + np.exp( -(X @ betas) )  )
    W = np.diag(pi * (1 - pi))
    cov = np.linalg.pinv(-hessian)

    # build hat matrix
    rootW = np.sqrt(W)
    H = X.T @ rootW.T
    H = cov @ H
    H = (rootW @ X) @ H

    # penalised score
                        # h_i
    U = X.T @ (y - pi + np.diag(H) * (.5 - pi))
    new_betas = betas + cov @ U
        
    return new_betas


def firth_logistic_regression(y : np.array,
                              X : np.ndarray,
                              max_iter : int = 1000,
                              tol : float = 1e-6,
                              test : str = 'lrt',
                              ) -> tuple[np.array, np.array, float]:
    """
    Perform Firth logistic regression.

    beta_hat = argmin | sum(y_i  - pi_i + h_i(1/2 - pi_i))x_{ir} |
        i -> N, r -> p = len(beta)

    Where: 
        h_i = ith diagonal element of:
            W^(1/2)X(X'WX)^(-1)X'W^(1/2)

        |I(beta)|^(1/2) = (X'WX)^(1/2) = Fisher information matrix

        pi_i = 1 / (1 + exp(-x_i*beta)) = probability of success
        W = diag(pi_i(1-pi_i)) = weight matrix
        X = design matrix (n x p) 
    """
    X = sm.add_constant(X)
    logit_model = sm.Logit(y, X)

    # fit null model
    null_model = sm.Logit(y, np.ones((len(y), 1)))
    null_result = null_model.fit(disp=0)
    start_vec = np.zeros(X.shape[1])
    start_vec[0] = null_result.params[0]
    betas = start_vec

    H = logit_model.hessian(betas)
    ll = logit_model.loglike(betas)

    conv = False
    for i in range(max_iter):
        new_betas = fit_firth_numba(betas, H, y, X)
        ll_next = logit_model.loglike(betas)
        H_next = logit_model.hessian(betas)

        while firth_likelihood(ll_next, H_next) < firth_likelihood(ll, H):
            new_betas = betas + 0.5 * (new_betas - betas)
            ll, H = ll_next, H_next
            ll_next = logit_model.loglike(new_betas)
            H_next = logit_model.hessian(new_betas)
            conv = (np.linalg.norm(new_betas - betas) < tol)
            if conv: break
        
        betas = new_betas
        if conv: break

    if new_betas is None:
        sys.stderr.write('Firth regression failed\n')
        return None

    # Calculate stats
    fitll = ll_next

    # add small value to the hessian to ensure it is invertible
    H_next += np.eye(H_next.shape[0]) * 1e-6
    bse = np.sqrt(np.diag(np.linalg.pinv(-H_next)))

    if test == 'null_model':
        return None, None, fitll, None, None
    
    elif test == 'wald':
        # Wald test for each coefficient
        stats = abs(betas / bse)
        pvals = 2 * ( 1 - norm.cdf(stats) )

    elif test == 'lrt':
        stats, pvals = [], [] # store chi2 stats and pvals
        # Likelihood ratio test
        for i, bse in enumerate(bse):
            null_X = np.delete(X, i, axis=1)

            _, _, null_fitll, _, _ = \
                firth_logistic_regression(
                    y, null_X, max_iter, tol, test='null_model')
            
            lr = -2 * (null_fitll - fitll)
            stats.append(lr)     
            lr_pval = 1 if lr < 0 else chi2.sf(lr, 1)
            pvals.append(lr_pval)


    return betas, bse, fitll, stats, pvals

'''